# Context

One may filter different features in the neural signals. Here it is investigated which preprocessing steps are suitable in this respect.

# Imports

In [ ]:
from skimage import io
import skimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, uniform_filter
import pickle

In [ ]:
import imageio
from pathlib import Path
from matplotlib.pyplot import show
from argparse import ArgumentParser

from pyoptflow import HornSchunck, getimgfiles
from pyoptflow.plots import compareGraphs

In [ ]:
from PIL import Image
import os
from scipy.signal import argrelextrema
from skimage import exposure

In [ ]:
import matplotlib
import matplotlib.animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [ ]:
np.array(np.clip([300],0,255), dtype=np.uint8)

In [ ]:
import matplotlib.animation as animation
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

In [ ]:
apply_mask

In [ ]:
mask = np.array(Image.open("mask.png"))[:,:,0] == 255

In [ ]:
plt.imshow(apply_mask(np.array(frames[0:2], dtype=np.double), mask, nan=True)[0])

### Import our custom utility methods

In [ ]:
import sys
%reload_ext autoreload
%autoreload 2
sys.path.append('..')

from utils.visualization_tools import *
import utils.visualization_tools
from utils.data_transformations import *
import utils.data_transformations
from utils.diverse import *
import utils.diverse

The following modules are available

In [ ]:
print_module_methods(utils.diverse)

In [ ]:
print_module_methods(utils.visualization_tools)

In [ ]:
print_module_methods(utils.data_transformations)

# Load data and inspect a frame of the raw data

In [ ]:
from pathlib import Path
source_folder = os.path.join(Path(os.getcwd()).parent, "datasets/source_data")

In [ ]:
frames = skimage.io.imread(os.path.join(source_folder,"runstart16_X1.tif"))

In [ ]:
plt.imshow(exposure.equalize_adapthist(normalize(np.mean(frames[:100,:,:],axis=0))))

In [ ]:
frames.shape

In [ ]:
frames = frames[:1000,:,:]

# Preprocessing

Here I calculate the difference from pixelwise mean as well as a smoothed version that promised to increase the signal to noise ratio.

In [ ]:
mean = np.mean(frames,axis=0)#pixelwise mean

In [ ]:
difference = normalize(framewise_difference(frames, mean, bigdata=False))

Some pixels are always brighter even at baseline activity. As the depiction below reveals one reason for this is bloodflow. Another reason is arguably a random pixel bias due to the camera. As we are interested in neither of these signals we aim at removing them.

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(np.min(difference,axis=0))
ax[0].axis("off")
ax[0].set_title("Pixelwise minimum")
ax[1].imshow(np.quantile(difference,.02,axis=0))
ax[1].axis("off")
ax[1].set_title("2nd percentile per pixelvector")

In [ ]:
#smooth = normalize(gaussian_filter(difference, 1))#Contaminated with bloodflow signal; remove later
#smooth = normalize(gaussian_filter(substract_pixel_min(difference, quantile=.1), 1))
smooth = normalize(gaussian_filter(substract_pixel_min(difference), 1))

In [ ]:
heartbeat_filtered = remove_frequency_from_pixel_vectors(smooth,16,18)

#smooth = heartbeat_filtered#TODO REMOVE

In [ ]:
framewise_normalized = (np.array([normalize(frame) for frame in heartbeat_filtered]))

To retrieve the difference from the spreading slow waves the difference between the strongly smoothened signal and the original signal (smooth) is being computed. A 3D gaussian kernel is used in this respect. For the time dimension only one slice is considered to calculate the stongly smoothened signal as the impact of a beginning slow wave would otherwise manifest in the resulting signal.

In [ ]:
smoother = normalize(uniform_filter(heartbeat_filtered,[0,60,60]))#[1,10,10]))# [20,30,30]))

In [ ]:
#plt.imshow(uniform_filter(framewise_normalized,[1,50,50])[0])

In [ ]:
details = heartbeat_filtered-smoother

In [ ]:
lowpass = [lowpass_filter(im,"vertical") for im in details]
details = np.array([normalize(frame) for frame in details])

In [ ]:
details = normalize(details)
lowpass = normalize(lowpass)

In [ ]:
contrast_enhanced = clipped_adaptive(framewise_normalized,.9)

# Image enhancement and filtering

For visualization of the slow waves total activation

## Smooth shows spreading slow waves

In [ ]:
%%capture
fig, ax = plt.subplots(1, figsize=(10,10))

im = ax.imshow(smooth[0,:,:], vmin =.3, vmax=.5)#vmin=.25,vmax=.3)
startframe = 70
ani = matplotlib.animation.FuncAnimation(fig, lambda i: im.set_array(smooth[startframe+i]), frames=20).to_jshtml()

In [ ]:
HTML(ani)

# Filter the heartbeat & retrieve bloodflow

In [ ]:
heartbeat = gaussian_filter(np.mean(smooth, axis = (1,2))-gaussian_filter(np.mean(smooth, axis = (1,2)),3),1)

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(10,10))
ax[0].set_title("Filtered heartbeat signal")
for x in maxima(heartbeat, pre_smoothing=0):
    ax[0].axvline(x, c="lightgray")
ax[0].plot(heartbeat)

ax[1].plot(*fourier(heartbeat))
ax[1].set_xticks(np.arange(0, 100+1, 5))

ax[1].set_title("Frequencies (FFT)")
ax[2].plot(*remove_frequency(heartbeat, 16, 18)[1:])
ax[2].set_title("Filtered frequencies")

In [ ]:
heartbeat_in_space = normalize(heartbeat_filtered-smooth)

In [ ]:
heartbeat_peaks = maxima(heartbeat, pre_smoothing=0)

In [ ]:
%%capture
ani = show_video(heartbeat_in_space[:,120:180,80:140],n_frames=35)

In [ ]:
HTML(ani)

In [ ]:
ca = clipped_adaptive(heartbeat_in_space[:,120:180,80:140])[:,:-8,:-8]

In [ ]:
#Mean over a few heartbeats 
vessels = np.array([exposure.equalize_adapthist(np.mean(ca[i:200+i:12,:,:],axis=0))[:-8,:-8] for i in range(12)])

In [ ]:
%%capture
ani = show_video(vessels,ca, n_frames=12)

In [ ]:
np.save("bloodflow.npy", ca)

Adaptive histogram equalization (left) and clipping of equalized signal (right)

In [ ]:
HTML(ani)

In [ ]:
n_period = 12
out = np.zeros([n_period,heartbeat_in_space.shape[1],heartbeat_in_space.shape[2]])
for x in heartbeat_peaks[1:-1]:
    out[:,:,:] += heartbeat_in_space[x-n_period//2:x+n_period//2,:,:]
out/=len(heartbeat_peaks[1:-1])

In [ ]:
%%capture
ani = show_video(normalize(out),n_frames=12)

In [ ]:
HTML(ani)

In [ ]:
x_comp, y_comp = horn_schunck(heartbeat_in_space[:24])

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], details[1])
start = 0
frames = 22

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(y_comp[i],x_comp[i], heartbeat_in_space[i+1], fig=fig, ax=ax)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

In [ ]:
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames).to_jshtml()
HTML(ani)

Desired: A method of how one could average over vector fields.

# Use upper percentiles per frame only

In [ ]:
upper_percentiles = [np.quantile(img, .9) for img in smooth]
percentile_thresholded = smooth[:,:,:].copy()
epsilon = .1

In [ ]:
upper_percentile_heartbeat = np.mean(percentile_thresholded,axis=(1,2))
upper_percentile_heartbeat -= gaussian_filter(upper_percentile_heartbeat, 1)

In [ ]:
plt.plot(upper_percentile_heartbeat)

In [ ]:
for i, threshold in enumerate(upper_percentiles):
    percentile_thresholded[i][percentile_thresholded[i] < threshold] = threshold
    percentile_thresholded[i][percentile_thresholded[i] > threshold+epsilon] = threshold+epsilon
    percentile_thresholded[i] = normalize(percentile_thresholded[i])

In [ ]:
%%capture
vid = show_video(percentile_thresholded, n_frames = 100)

In [ ]:
HTML(vid)

## Framewise normalization of smoothened tensor shows details

For visualization of nuances of small scale travelling peaks in the activation by linear scaling mapping the lowest value to 0 and the highest to 1. 

In [ ]:
%%capture
import matplotlib.animation
from IPython.display import HTML
fig, ax = plt.subplots(1, figsize=(10,10))
def display(frame):
    global fig, ax
    ax.cla()
    im = ax.imshow(frame,vmin=0,vmax=1)#NORMALIZED FRAME HERE
    return fig, ax
startframe = 50
ani = matplotlib.animation.FuncAnimation(fig, lambda i: display(framewise_normalized[startframe+i]), frames=20).to_jshtml()

In [ ]:
HTML(ani)

## The difference to the strongly smoothened tensor (in space and time) improves details 

In [ ]:
%%capture
ani = show_video(details, lowpass, 20)

In [ ]:
HTML(ani)

## Adaptive histogram equalization

In [ ]:
background = normalize(np.mean(lowpass,axis=0))
img = contrast_enhanced[0]

In [ ]:
%%capture
startframe = 0
n_frames = 200
superimposed = [superimpose(a,b, cm.gray, cm.viridis) for a,b in zip(contrast_enhanced[startframe:startframe+n_frames],
                                                                          lowpass[startframe:startframe+n_frames])]
frame_selection = smooth[startframe:startframe+n_frames].copy()
frame_selection[frame_selection<.3] = .3
frame_selection[frame_selection>.5] = .5
frame_selection = normalize(frame_selection)

ani = show_video(superimposed, frame_selection, 200, orient="vertical")

In [ ]:
fig, ax = plt.subplots(2)
signal = np.mean(contrast_enhanced,axis=(1,2))
signal -= gaussian_filter(signal,1)
ax[0].plot(signal)
ax[1].plot(*fourier(signal))

In [ ]:
HTML(ani)

The signal contains mostly high frequency components. The peak in frequency at around 16 Hz that corresponds to the heartbeat is absent. Spatially the filtered clusters do not preferably overlap with the low frequency signal that is low-pass-filtered from the detail images (smooth minus smoother). Both can be considered as an indicator that the filtered signal does not predominantly correspond to blood that moves at a uniform speed.

In [ ]:
def sample_frame_in_roi(frame, window_size, left, right, top, bottom):   
    snippet = frame[left:right,top:bottom]
    further_preprocessed = exposure.equalize_adapthist(normalize(snippet), clip_limit=0.03)
    further_preprocessed = further_preprocessed[:window_size-8,:window_size-8]
    return further_preprocessed
    
def sample_roi(tensor, start_frame, stop_frame, window_size, left, top):
    right = left + window_size
    bottom = top + window_size
    return np.array([sample_frame_in_roi(tensor[i], window_size, left, right, top, bottom) for i in range(start_frame,stop_frame)])

In [ ]:
#np.save("roi_background.npy",sample_roi(frames,0,300, 60, 120, 80))

In [ ]:
window_size = 60
roi = sample_roi(details,0, 300, 60, 120, 80)

In [ ]:
np.save("roi.npy",roi)

In [ ]:
roi.shape

In [ ]:
%%capture
roi1 = clipped_adaptive(heartbeat_in_space[:300,120:120+window_size,80:80+window_size])[:,:52,:52]

clipped_roi = roi.copy()
clipped_roi[clipped_roi>.8] = .8
clipped_roi[clipped_roi<.6] = .6
clipped_roi = normalize(clipped_roi)
ani = show_video(clipped_roi,roi1, 200)

In [ ]:
HTML(ani)

In [ ]:
HTML(show_video([normalize(np.log(f)) for f in heartbeat_in_space[:300,120:120+window_size,80:80+window_size]],
               [normalize(f) for f in smooth[:300,120:120+window_size,80:80+window_size]]))

In [ ]:
print(np.corrcoef(heartbeat_in_space[:300,120:120+window_size,80:80+window_size].flatten(),
            details[:300,120:120+window_size,80:80+window_size].flatten()))

In [ ]:
roi.flatten

In [ ]:
roi1.shape

In [ ]:
plt.imshow(heartbeat_in_space[2])

In [ ]:
np.corrcoef(roi.flatten(), gaussian_filter(roi1,1).flatten())

In [ ]:
corr_of_means = np.corrcoef(np.mean(clipped_roi,axis=0),np.mean(roi1,axis=0))
corr_of_means = corr_of_means[len(corr_of_means)//2:,:len(corr_of_means)//2]
plt.imshow(corr_of_means)
plt.colorbar()
print(np.mean(np.abs(corr_of_means)))

In [ ]:
HTML(ani)

#### Are there signs for systematic noise or artifacts?

In [ ]:
upper_decentile_roi = np.array([np.quantile(f,0.9) for f in roi])

In [ ]:
signal = upper_decentile_roi - gaussian_filter(upper_decentile_roi, 5)
x, freq = fourier(signal)
fig, ax = plt.subplots(2)
ax[0].plot(signal-np.mean(signal))
ax[1].plot(x,freq)

Fourier plot does not indicate any dominant frequencies

# Horn and Schunck dense optical flow

In [ ]:
x_comp, y_comp = horn_schunck(contrast_enhanced, 200)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], details[1])
start = 25
frames = 10

def animate(i):
    global start
    i += start
    print(".", end ="")    
    display_combined(y_comp[i]/10,x_comp[i]/10, details[i+1], fig=fig, ax=ax)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
roi = sample_roi(details,0,100)

In [ ]:
x_comp, y_comp = horn_schunck(roi, len(roi)-1)

In [ ]:
%%capture
fig, ax = display_combined(x_comp[0],y_comp[0], details[1])
start = 0
frames = 10

def animate(i):
    i += start
    print(".", end ="")    
    display_combined(x_comp[i]/5,y_comp[i]/5, roi[i+1], fig=fig, ax=ax, scale=10, quivstep=1)
    #Q.set_UVC(np.flipud(rescaled[:,:,0]), -np.flipud(rescaled[:,:,1]))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frames)

In [ ]:
HTML(ani.to_jshtml())

# Conclusion

One can filter small scale motion patterns and largescale dynamics. The big size of the data represents a challange becuase of working memory restrictions when using NumPy methods directly. Custom methods can help to reduce the memory requirements. Developing scripts that run in a computational grid on computers with large memory capacities could also help.